In [37]:
# Import all dependencies

import numpy as np
import os
from netCDF4 import Dataset
from numpy import sum 

def average_stratospheric_aerosol_optical_depth(dataset_path):
    """
    Calculate the average stratospheric optical depth (SAOD) by integrating extinction over
    stratospheric layers, and then averaging over time and latitude.
    """
    # Read in the CMIP7 dataset
    ext_set = Dataset(dataset_path, mode="r")
    
    # Find WAVELENGTH_INDEX in wavelength such that wavelength[WAVELENGTH_INDEX] is 550 nm as prescribed
    wavelengths = ext_set["wavelength"][:]
    WAVELENGTH = 550.0 * 1e-9
    WAVELENGTH_INDEX = wavelengths.searchsorted([WAVELENGTH])
    assert wavelengths[WAVELENGTH_INDEX] == WAVELENGTH
    
    # Select the ext values for the prescribed wavelength 550 nm, setting all NaN values to 0
    ext_var = ext_set["ext"]
    exts_at_wl = np.nan_to_num(ext_var[:, :, :, WAVELENGTH_INDEX]).reshape(np.shape(ext_var)[:-1])
    
    # Calculate the thickness of each atmospheric layer
    height_bnds = ext_set["height_bnds"][:]
    height_diffs = height_bnds[:,1] - height_bnds[:,0]
    
    # Calculate the SAOD values for each month and latitude band by quadrature of ext over the heights of the atmosphere layers 
    ext_dims = ext_var.dimensions
    EXT_HEIGHT_AXIS = ext_dims.index("height")
    
    # The dot product works because height is the last axis
    saods_at_wl = np.ma.dot(exts_at_wl, height_diffs)
    
    # Create weights corresponding to the proportion of days in each month of the year
    times = np.ma.append(ext_set["time"][:],[365])
    month_days = np.diff(times)
    MONTH_WEIGHTS = np.array(month_days) / sum(month_days)
    assert len(MONTH_WEIGHTS) == 12
    
    # Take the time average of the saod values for each latitude band
    weighted_saod_at_each_lat = np.ma.dot(MONTH_WEIGHTS, saods_at_wl)
    
    # Calculate the difference between latitudes
    lats = ext_set["lat"][:]
    lat_bnds = ext_set["lat_bnds"][:]
    lat_diffs = lat_bnds[:,1] - lat_bnds[:,0]
    
    # Create weights corresponding to the relative area of each latitude band
    sindeg = lambda t: np.sin(np.radians(t))
    lat_weights = [(sindeg(lats[k] + lat_diffs[k] / 2.0) - sindeg(lats[k] - lat_diffs[k] / 2.0)) / 2.0 for k in range(len(lats))]
    assert sum(lat_weights) == 1.0
    
    # Calculate the average SAOD by weighting by the relative area of each latitude band
    return np.ma.dot(lat_weights, weighted_saod_at_each_lat)


# Specify the CMIP7 dataset

INPUT4MIPS_DIR = "/g/data/qv56/replicas/input4MIPs"
EXT_FILE = "CMIP7/CMIP/uoexeter/UOEXETER-CMIP-2-0-0/atmos/monC/ext/gnz/v20250227/ext_input4MIPs_aerosolProperties_CMIP_UOEXETER-CMIP-2-0-0_gnz_185001-202112-clim.nc"
EXT_PATH = os.path.join(INPUT4MIPS_DIR, EXT_FILE)

# Calculate and print the SAOD
volcts_val = average_stratospheric_aerosol_optical_depth(EXT_PATH) * 1.0e4
print(f'VOLCTS_val = {volcts_val:.2f}')

VOLCTS_val = 141.17
